In [2]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns

In [13]:
from pyspark.sql import SparkSession
# Spark 세션 생성
spark = (
    SparkSession.builder
    .appName("olist")
    .config("spark.driver.memory", "6g")
    .config("spark.executor.memory", "6g")
    .config("spark.sql.shuffle.partitions", "8")
    .getOrCreate()
)

orders = spark.read.csv('/app/spark/Olist/fact_table/fact_orders.csv',header=True,inferSchema=True)
items=spark.read.csv('/app/spark/Olist/fact_table/fact_items.csv',header=True,inferSchema=True)
order_items = orders.join(items, on='order_id')



In [14]:
order_items.show(5)

+--------------------+--------------------+------------+------------------------+-----------------+----------------------------+-----------------------------+-----------------------------+----------------+--------------------------+--------------------+--------------+--------------------+--------------------+-------------------+------+-------------+-----+
|            order_id|         customer_id|order_status|order_purchase_timestamp|order_approved_at|order_delivered_carrier_date|order_delivered_customer_date|order_estimated_delivery_date|days_to_delivery|estimated_days_to_delivery|delay_flag_estimated|delay_flag_avg|          product_id|           seller_id|shipping_limit_date| price|freight_value|count|
+--------------------+--------------------+------------+------------------------+-----------------+----------------------------+-----------------------------+-----------------------------+----------------+--------------------------+--------------------+--------------+----------------

### Seller 와 Customer 사이의 거리 계산

In [18]:
sellers = spark.read.csv('/app/spark/Olist/dim_table/dim_sellers.csv',header=True,inferSchema=True)
order_items = order_items.join(sellers,on='seller_id')
order_items.show(5)

+--------------------+--------------------+--------------------+------------+------------------------+-----------------+----------------------------+-----------------------------+-----------------------------+----------------+--------------------------+--------------------+--------------+--------------------+-------------------+------+-------------+-----+----------------------+-----------+------------+-------------+-------------------+------------------+
|           seller_id|            order_id|         customer_id|order_status|order_purchase_timestamp|order_approved_at|order_delivered_carrier_date|order_delivered_customer_date|order_estimated_delivery_date|days_to_delivery|estimated_days_to_delivery|delay_flag_estimated|delay_flag_avg|          product_id|shipping_limit_date| price|freight_value|count|seller_zip_code_prefix|seller_city|seller_state|seller_region|         seller_lat|        seller_lng|
+--------------------+--------------------+--------------------+------------+-----

In [20]:
customers = spark.read.csv('/app/spark/Olist/dim_table/dim_customers.csv',inferSchema=True,header=True)
mart_overall = order_items.join(customers,on='customer_id')
mart_overall.show(5)

25/03/17 13:43:40 WARN SparkStringUtils: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


+--------------------+--------------------+--------------------+------------+------------------------+-----------------+----------------------------+-----------------------------+-----------------------------+----------------+--------------------------+--------------------+--------------+--------------------+-------------------+-----+-------------+-----+----------------------+-----------+------------+-------------+-------------------+------------------+--------------------+------------------------+-------------+--------------+------------------+------------------+---------------+
|         customer_id|           seller_id|            order_id|order_status|order_purchase_timestamp|order_approved_at|order_delivered_carrier_date|order_delivered_customer_date|order_estimated_delivery_date|days_to_delivery|estimated_days_to_delivery|delay_flag_estimated|delay_flag_avg|          product_id|shipping_limit_date|price|freight_value|count|seller_zip_code_prefix|seller_city|seller_state|seller_re

In [21]:
col = ['seller_lat', 'seller_lng', 'customer_lat', 'customer_lng']

In [22]:
from geopy.distance import geodesic

def calculate_distance(x):
    seller_coords = (x['seller_lat'], x['seller_lng'])
    customer_coords = (x['customer_lat'], x['customer_lng'])
    return geodesic(seller_coords, customer_coords).kilometers 

In [ ]:
mart_overall = mart_overall.toPandas()

In [23]:
mart_overall['distance'] = mart_overall[col].apply(lambda x: calculate_distance(x), axis=1)
mart_overall['distance']

AttributeError: 'DataFrame' object has no attribute 'apply'

In [27]:
overall_joined.drop(columns=['seller_lat', 'seller_lng', 'customer_lat', 'customer_lng'], inplace=True)
overall_joined.head()

,order_id,customer_id,order_status,order_purchase_timestamp,order_approved_at,order_delivered_carrier_date,order_delivered_customer_date,order_estimated_delivery_date,days_to_delivery,estimated_days_to_delivery,...,seller_city,seller_state,seller_region,season,customer_unique_id,customer_zip_code_prefix,customer_city,customer_state,customer_region,distance
0,e481f51cbdc54678b7cc49136f2d6af7,9ef432eb6251297304e76186b10a928d,delivered,2017-10-02 10:56:33,2017-10-02 11:07:15,2017-10-04 19:55:00,2017-10-10 21:25:13,2017-10-18,8,15,...,maua,SP,남동부,rainy,7c396fd4830fd04220f754e42b4e5bff,3149,sao paulo,SP,남동부,18.566632
1,53cdb2fc8bc7dce0b6741e2150273451,b0830fb4747a6c6d20dea0b8c802d7ef,delivered,2018-07-24 20:41:37,2018-07-26 03:24:27,2018-07-26 14:31:00,2018-08-07 15:27:45,2018-08-13,12,17,...,belo horizonte,MG,남동부,dry,af07308b275d755c9edb36a90c618231,47813,barreiras,TO,북부,856.076259
2,53cdb2fc8bc7dce0b6741e2150273451,b0830fb4747a6c6d20dea0b8c802d7ef,delivered,2018-07-24 20:41:37,2018-07-26 03:24:27,2018-07-26 14:31:00,2018-08-07 15:27:45,2018-08-13,12,17,...,belo horizonte,MG,남동부,dry,af07308b275d755c9edb36a90c618231,47813,barreiras,BA,북동부,961.437029
3,47770eb9100c2d0c44946d9cf07ec65d,41ce2a54c0b03bf3443c3d931a367089,delivered,2018-08-08 08:38:49,2018-08-08 08:55:23,2018-08-08 13:50:00,2018-08-17 18:06:29,2018-09-04,9,26,...,guariba,SP,남동부,dry,3a653a41f6f9fc3d2a113cf8398680e8,75265,vianopolis,GO,중서부,512.100044
4,949d5b44dbf5de918fe9c16f97b45f8a,f88197465ea7920adcdbec7375364d82,delivered,2017-11-18 19:28:06,2017-11-18 19:45:59,2017-11-22 13:39:59,2017-12-02 00:28:42,2017-12-15,13,26,...,belo horizonte,MG,남동부,rainy,7c142cf63193a1473d2e66489a9ae977,59296,sao goncalo do amarante,RN,북동부,1816.085655


In [ ]:
# overall_joined.to_csv('mart_orders.csv', index=False)